In [1]:
%load_ext autoreload
%autoreload 2

In [354]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arctic
from functions import generate_volumebars, get_tick_data, generate_dollarbars, generate_tickbars, fix_timestamps
from datetime import datetime
# from statsmodels.tsa.stattools import adfuller, coint
# from statsmodels.stats.stattools import jarque_bera
import statsmodels.api as sm
from tqdm.notebook import tqdm
import seaborn as sns
import math
pd.set_option('float_format', '{:f}'.format)

In [4]:
start_date = datetime(2020,7,1)
end_date = datetime(2020,7,30)

%time df = get_tick_data('binance_futures', 'BTCUSDT', start_date, end_date)
%time df = fix_timestamps(df)

CPU times: user 8.47 s, sys: 1.97 s, total: 10.4 s
Wall time: 11.7 s
CPU times: user 2.07 s, sys: 858 ms, total: 2.93 s
Wall time: 2.92 s


## EXERCISES
### 18.1 Form dollar bars on E-mini S&P 500 futures:

In [5]:
dbars = generate_dollarbars(df, 10000)

#### (a) Quantize the returns series using the binary method.

In [18]:
r = dbars.close.pct_change().dropna()
be = r[r!=0]
be = pd.Series(np.where(be > 0,1,0), index=be.index)

#### (b) Quantize the returns series using the quantile encoding, using 10 letters.

In [126]:
qe = r.copy()

quantiles = [round(x,1) for x in np.linspace(0.1,1,10)]
for q in quantiles:
    qe[qe <= r.quantile(q)] = q
qe = (qe*10).astype(int) - 1

qe.value_counts()

4    306685
9    218502
7    218502
2    218502
0    218502
8    218501
3    218501
1    218501
6    218496
5    130323
Name: close, dtype: int64

#### (c) Quantize the returns series using the sigma encoding, where 𝜎 is the standard deviation of all bar returns.

In [143]:
se = r.copy()

sigma = np.std(r)
minr = min(r)
maxr = max(r)

num_codes = math.ceil((maxr-minr)/sigma)

for c in range(1, num_codes+1):
    l = minr + sigma * (c-1)
    u = minr + sigma * c
    se[(se >= l) & (se < u)] = c
se = se.astype(int)

In [144]:
se.value_counts()

107    1409448
108     430236
106     184754
109      73811
105      36216
        ...   
148          1
152          1
156          1
182          1
1            1
Name: close, Length: 94, dtype: int64

#### (d) Compute the entropy of the three encoded series, using the plug-in method.

In [153]:
def pmf1(msg,w):
    # Compute the prob mass function for a one-dim discrete rv
    # len(msg)-w occurrences
    lib={}
    if not isinstance(msg, str):
        msg = ''.join(map(str, msg))
#     print(msg)
    for i in range(w, len(msg)+1):
        msg_ = msg[i-w:i]
#         print(msg_)
        if msg_ not in lib:
            lib[msg_] = [i - w]
        else:
            lib[msg_] = lib[msg_] + [i-w] 
#     print(lib)
    pmf = float(len(msg) - w +1) 
#     print(pmf)
    pmf = {i:len(lib[i]) / pmf for i in lib} 
    return pmf

def plugIn(msg,w):
    # Compute plug-in (ML) entropy rate
    pmf = pmf1(msg, w)
    out = -sum([pmf[i] * np.log2(pmf[i]) for i in pmf])/w 
    return out, pmf
# Entropy binary encoding message lenght =1
print(plugIn(be[0:100000], 1))
# Entropy quantile encoding message lenght =1
print(plugIn(qe[0:100000], 1))

(0.9999885735365703, {'1': 0.50199, '0': 0.49801})
(3.2706207667954335, {'5': 0.05199, '3': 0.1168, '7': 0.11336, '2': 0.11469, '4': 0.13614, '1': 0.09978, '9': 0.07071, '0': 0.07092, '6': 0.1254, '8': 0.10021})


In [159]:
# Entropy for quantile encoding for 
pmf = (se.value_counts() / se.value_counts().sum()).to_dict()
out = -sum([pmf[i] * np.log2(pmf[i]) for i in pmf])/1
out, pmf

(1.6316712373028968,
 {107: 0.6450518646325083,
  108: 0.1969029960892717,
  106: 0.08455502593803704,
  109: 0.033780546128973944,
  105: 0.016574714590060022,
  110: 0.008057152925723622,
  104: 0.00466907549833754,
  111: 0.002851239007512534,
  103: 0.0018471269075956,
  112: 0.001280998070951458,
  102: 0.0009276824186561648,
  113: 0.000643931506191033,
  101: 0.000519904897678048,
  114: 0.00038535204563813064,
  100: 0.0003029727484708343,
  115: 0.0002407306128333215,
  99: 0.00022013578854149742,
  116: 0.0001560630018558225,
  98: 0.00013501051480195787,
  117: 0.00010938140012768792,
  97: 0.00010388944698320149,
  118: 8.237929716729634e-05,
  96: 7.27683791644451e-05,
  119: 6.681876325791814e-05,
  95: 6.453044944771546e-05,
  120: 5.400420592078315e-05,
  94: 3.707068372528335e-05,
  121: 3.524003267712121e-05,
  93: 3.157873058079693e-05,
  122: 2.1510149815905154e-05,
  92: 2.1052487053864617e-05,
  123: 1.9679498767743014e-05,
  124: 1.4645208385297126e-05,
  90: 1.4

In [161]:
for w in range(1,10):
    print(w)
    print('BE: {}'.format(plugIn(be[0:100000], w)[0]))
    print('QE: {}'.format(plugIn(qe[0:100000], w)[0]))

1
BE: 0.9999885735365703
QE: 3.2706207667954335
2
BE: 0.9734101557287611
QE: 3.2153931789504084
3
BE: 0.9586068963054594
QE: 3.179469040668463
4
BE: 0.9489920470496829
QE: 3.1390870757901386
5
BE: 0.9422235678513804
QE: 3.001404347644249
6
BE: 0.937195629415263
QE: 2.685854402911815
7
BE: 0.9333441463125796
QE: 2.348410713348405
8
BE: 0.9302397901424408
QE: 2.0673041771141114
9
BE: 0.927635149456782
QE: 1.841668750488327


In [235]:
def lempelZiv_lib(msg): 
    i=1
    lib=[msg[0]] 
    while i<len(msg):
        for j in range(i,len(msg)):
            msg_=msg[i:j+1]
            if msg_ not in lib: 
                lib.append(msg_) 
                break
        i=j+1 
    return lib

def matchLength(msg,i,n):
    # Maximum matched length+1, with overlap. 
    # i>=n & len(msg)>=i+n
    subS=''
    for l in range(n):
        msg1=msg[i:i+l+1]
#         print('-'+msg1)
        for j in range(i-n,i):
            msg0=msg[j:j+l+1] 
#             print(msg0)
            if msg1==msg0:
                subS=msg1
                break # search for higher l. 
    return len(subS)+1,subS # matched length + 1

# matchLength('10000111', 6, 2)

In [475]:
def konto(msg, window=None): 
    '''
    * Kontoyiannis’ LZ entropy estimate, 2013 version (centered window). 
    * Inverse of the avg length of the shortest non-redundant substring. 
    * If non-redundant substrings are short, the text is highly entropic. 
    * window == None for expanding window, in which case len(msg) % 2 == 0
    * If the end of msg is more relevant, try konto(msg[::-1]) '''
    
    out={'num':0, 'sum':0, 'subS':[]}
    if not isinstance(msg, str):
        msg = ''.join(map(str, msg))
    if window is None:
        points = range(1, len(msg) // 2 + 1) # 1 -7
    else:
        window = min(window, len(msg) // 2) 
        points = range(window, len(msg) - window + 1)
        
    for i in points:
        if window is None:
            l, msg_ = matchLength(msg, i, i)
#             print(l, msg_)
            out['sum'] += np.log2(i + 1) / l # to avoid Doeblin condition 
        else:
            
            l , msg_ = matchLength(msg, i, window)
            out['sum'] += np.log2(window + 1) / l # to avoid Doeblin condition 
        out['subS'].append(msg_)
        out['num'] += 1
    out['h'] = out['sum']/out['num'] 
    out['r'] = 1 - out['h'] / np.log2(len(msg)) # redundancy, 0<=r<=1 
    return out

# print(konto(msg*2))
# print(konto(msg+msg[::-1]))

In [491]:
print(konto('10000111'))
print(konto('10000110'))

{'num': 26, 'sum': 7.81829058499836, 'subS': ['0', '00', '000', '0000', '00000', '000000', '0000000', '00000000', '000000000', '0000000000', '00000000000', '000000000000', '0000000000000', '00000000000000', '000000000000000', '0000000000000000', '00000000000000000', '000000000000000000', '0000000000000000000', '00000000000000000000', '000000000000000000000', '0000000000000000000000', '00000000000000000000000', '000000000000000000000000', '0000000000000000000000000', '00000000000000000000000000'], 'h': 0.30070348403839847, 'r': 0.9472490721932487}
{'num': 4, 'sum': 3.3559515476840662, 'subS': ['', '00', '00', '0'], 'h': 0.8389878869210166, 'r': 0.7203373710263279}


In [222]:
print(konto('10000111'[::-1]))
print(konto('10000110'[::-1]))

{'num': 4, 'sum': 3.8729632740824185, 'subS': ['1', '1', '', '000'], 'h': 0.9682408185206046, 'r': 0.6772530604931317}
{'num': 4, 'sum': 3.3729632740824185, 'subS': ['', '1', '0', '000'], 'h': 0.8432408185206046, 'r': 0.7189197271597985}


#### (e) Compute the entropy of the three encoded series, using Kontoyiannis’ method, with a window size of 100.

In [238]:
win=100
# Entropy binary encoding
%time print(konto(be[0:10000], win)['h'])
# Entropy quantile encoding
%time print(konto(qe[0:10000], win)['h'])
# Entropy sigma encoding 
%time print(konto(se[0:10000], win)['h'])

0.8312900405921321
CPU times: user 17 s, sys: 36 ms, total: 17 s
Wall time: 17.1 s
2.4797323127529904
CPU times: user 17.7 s, sys: 73.8 ms, total: 17.8 s
Wall time: 18 s
0.6370205498768906
CPU times: user 48.4 s, sys: 190 ms, total: 48.6 s
Wall time: 49.1 s


### 18.2 Using the bars from exercise 1:
#### (a) Compute the returns series, {rt}.
#### (b) Encode the series as follows: 0 if rtrt−1 < 0, and 1 if rtrt−1 ≥ 0.

In [257]:
rt = pd.concat([r.rename('or'), r.shift(1)], axis=1)
rt = rt['or'] * rt.close
rt = rt.dropna()
rt = pd.Series(np.where(rt < 0,0,1), index=rt.index)
# rt.iloc[0:1].values

#### (c) Partition the series into 1000 non-overlapping subsets of equal size (you may have to drop some observations at the beginning).

In [263]:
sub_size=1000
num = rt.shape[0]//1000

s = []
for i in range(num):
    s_ = rt.iloc[i:1000 * (i+1)]
    s.append(s_)

#### (d) Compute the entropy of each of the 1000 encoded subsets, using the plug-in method.

In [273]:
len(s)

2185

In [283]:
%time pim = [plugIn(x, 1)[0] for x in s[0:10]]
pim

CPU times: user 300 ms, sys: 6.64 ms, total: 307 ms
Wall time: 315 ms


[0.8988610370442902,
 0.8839044851478619,
 0.8835622497868979,
 0.8948786178658559,
 0.8946099563127436,
 0.8830171669957383,
 0.880380157577554,
 0.8829818354221756,
 0.8867138220301641,
 0.8858546770418745]

#### (e) Compute the entropy of each of the 1000 encoded subsets, using the Kontoyiannis method, with a window size of 100.

In [279]:
%time kom = [konto(x, 100)['h'] for x in s[0:10]]
kom

CPU times: user 1min 31s, sys: 499 ms, total: 1min 31s
Wall time: 1min 33s


[0.7863022729838257,
 0.7789841670451053,
 0.7871440961029277,
 0.7931576293177454,
 0.7948583369809137,
 0.7859008256896164,
 0.7850257988329868,
 0.7887369258224388,
 0.789404525826153,
 0.7883841362470622]

#### (f) Compute the correlation between results 2.d and 2.e.

In [284]:
np.corrcoef(np.array([pim, kom]))
# .shape

array([[1.        , 0.51669967],
       [0.51669967, 1.        ]])

### 18.3 Draw 1000 observations from a standard Normal distribution:
#### (a) What is the true entropy of this process?

In [312]:
nd = pd.Series(np.random.randn(1000)*10)

In [462]:
np.random.seed(0)
l = []
pl = []
for i in range(100):
    nd = pd.Series(np.random.randn(100))
    end = get_quantile_encoding(nd, 9)
    l.append(konto(end,2)['h'])
    pl.append(plugIn(end,4)[0])
print(np.mean(l))
print(np.std(l))

print(np.mean(pl))
print(np.std(pl))

1.4229804224687623
0.029352307861268093
1.647916354876682
0.003571238778492555


In [439]:
(0.5)*np.log(2*np.pi*np.e*1**2)

1.4189385332046727

#### (b) Label the observations according to 8 quantiles.

In [470]:
def get_quantile_encoding(series, num_q):
    quantiles = [series.quantile(x) for x in np.linspace(1/num_q, 1, num_q)]

    qe = series.copy()
    for code, limit in reversed(list(enumerate(quantiles))):
        qe.loc[series<=limit] = code
    return qe.astype(int)

end = get_quantile_encoding(nd, 10)

#### (c) Estimate the entropy using the plug-in method.

In [471]:
plugIn(end,5)[0]

1.3169925001442322

#### (d) Estimate the entropy using the Kontoyiannis method:
#### (i) using a window size of 10.

In [472]:
konto(end, 2)['h']

1.495093492948309

In [473]:
konto(end, 10)['h']

2.2778150575389633

#### (ii) using a window size of 100.

In [476]:
konto(end, 100)['h']

1.8908084473238318

### 18.4 Using the draws from exercise 3, {xt}: t=1,...,1000:
#### (a) Compute yt =𝜌yt−1 +x_t, where 𝜌=.5, y0 =0.

In [350]:
p=0.5
yt=[0]
for x in nd:
    y = p*yt[-1] + x
    yt.append(y)
# this turns into a autocorrelated time series

In [358]:
# sm.graphics.tsa.plot_acf(yt, lags=120,
#                                  alpha=0.05, unbiased=True, fft=True,
#                                  zero=False)

#### (b) Label {yt} the observations according to 8 quantiles.

In [370]:
qe = get_quantile_encoding(pd.Series(yt), 8)

#### (c) Estimate the entropy using the plug-in method.

In [477]:
plugIn(qe, 1)[0]

3.321921632906739

#### (d) Estimate the entropy using the Kontoyiannis method
#### (i) using a window size of 10. 

In [482]:
konto(qe,10)['h']

2.249804833043859

#### (ii) using a window size of 100.

In [373]:
konto(qe,100)['h']

2.483130990637225

### 18.5 Suppose a portfolio of 10 holdings with equal dollar allocations.
#### (a) The portion of the total risk contributed by the ith principal component is 1/10 , i = 1, ... , 10. What is the portfolio’s entropy?


#### (b) The portion of the total risk contributed by the ith principal component is 1 -i/55, i = 1, ... , 10. What is the portfolio’s entropy?


#### (c) The portion of the total risk contributed by the ith principal component is 𝛼1/10 +(1−𝛼)(1−i/55), i=1,...,10, 𝛼 ∈ [0,1]. Plot the portfolios entropy as a function of 𝛼.

In [483]:
"""
Simple script implementing Kaspar & Schuster's algorithm for
Lempel-Ziv complexity (1976 version).

If you use this script, please cite the following paper containing a sample
use case and further description of the use of LZ in neuroscience:

Dolan D. et al (2018). The Improvisational State of Mind: A Multidisciplinary
Study of an Improvisatory Approach to Classical Music Repertoire Performance.
Front. Psychol. 9:1341. doi: 10.3389/fpsyg.2018.01341

Pedro Mediano and Fernando Rosas, 2019
"""
import numpy as np

def LZ76(ss):
    """
    Calculate Lempel-Ziv's algorithmic complexity using the LZ76 algorithm
    and the sliding-window implementation.

    Reference:

    F. Kaspar, H. G. Schuster, "Easily-calculable measure for the
    complexity of spatiotemporal patterns", Physical Review A, Volume 36,
    Number 2 (1987).

    Input:
      ss -- array of integers

    Output:
      c  -- integer
    """

    ss = ss.flatten().tolist()
    i, k, l = 0, 1, 1
    c, k_max = 1, 1
    n = len(ss)
    while True:
        if ss[i + k - 1] == ss[l + k - 1]:
            k = k + 1
            if l + k > n:
                c = c + 1
                break
        else:
            if k > k_max:
               k_max = k
            i = i + 1
            if i == l:
                c = c + 1
                l = l + k_max
                if l + 1 > n:
                    break
                else:
                    i = 0
                    k = 1
                    k_max = 1
            else:
                k = 1
    return c


if __name__ == '__main__':
    # Simple string, low complexity
    ss = np.array([0,0,0,0,0,0,0,1,1,1,1,1,1])
    print('The complexity of the string %s is %i'%(ss, LZ76(ss)))

    # Irregular string, high complexity
    ss = np.array([0,1,1,0,1,0,0,1,0,1,1,1,0])
    print('The complexity of the string %s is %i'%(ss, LZ76(ss)))

The complexity of the string [0 0 0 0 0 0 0 1 1 1 1 1 1] is 3
The complexity of the string [0 1 1 0 1 0 0 1 0 1 1 1 0] is 6


In [488]:
konto([0,1,1,0,1,0,0,1,0,1,1,1,0])

{'num': 6,
 'sum': 5.227443929531345,
 'subS': ['', '1', '01', '10', '0', '010'],
 'h': 0.8712406549218908,
 'r': 0.7645575333518588}